In [2]:
import pandas as pd
from ast import literal_eval

In [3]:
class ProximalCues:
    def __init__(self):
        generic = lambda x: literal_eval(x)

        conv = {'pc': generic}

        self.urls = pd.read_csv("../data/urls/url-reference_new.csv", converters=conv)

    # Get the proximal cues by page id
    def get_proximal_cues_by_id(self, id):
        return self.urls.loc[self.urls.id==id].pc.iloc[0]

In [4]:
import numpy as np


class TfIdf:
    def __init__(self):
        # Loading in the tf-idf
        # Rows are documents, the first column is the document id
        # Columns are keywords, the first row is the keyword id
        self.tfidf = pd.read_csv("../data/tf_idf/tf_idf.csv")

        # Loading in the keywords
        # Two columns, column 1 is id, column 2 is keyword
        # We can access the weight in the tf-idf by first accessing the id number from the keyword file
        self.tfidf_keywords = pd.read_csv("../data/tf_idf/tf_idf_keywords.csv")
        self.tfidf_keywords.columns = ["id", "keyword"]

    def get_id_by_keyword(self, keyword):
        return self.tfidf_keywords.id.iloc[self.tfidf_keywords[self.tfidf_keywords.keyword == keyword].index].values[0]


    def get_keyword_by_id(self, id):
        return self.tfidf_keywords.loc[self.tfidf_keywords.id==id, 'keyword'].iloc[0]
    # Method to get the tfidf value of a keyword in a page
    def get_tf_idf_value(self, page_id, keyword):
        keyword_id = self.get_id_by_keyword(keyword)
        return self.tfidf.iloc[page_id, keyword_id + 1]

    def get_all_keywords_by_id(self, page_id):
        keywords = self.tfidf.iloc[page_id][1:].values
        return keywords

    def get_all_keywords_by_id_normalized(self, page_id):
        keywords_weights = self.get_all_keywords_by_id(page_id)
        sum_value = sum(keywords_weights)
        keywords_weights_normalized = keywords_weights/sum_value
        return keywords_weights_normalized

    def get_number_of_keywords(self):
        return self.tfidf_keywords.shape[0]

In [5]:
class AdjacencyMatrix:
    def __init__(self):
        # Loading in the adjacency matrix
        # First column and row are data immediately, so access directly by page id
        # If row(page_id_1) leads to column(page_id_2) = 1 else = 0
        self.adjacency_matrix = pd.read_csv("../data/matrices/adjacency_matrix.csv", header=None)

    # Method to get the adjacency value from two page ids
    def get_adjacency_value(self, page_id_1, page_id_2):
        return self.adjacency_matrix.iloc[page_id_1, page_id_2] == 1

In [6]:
adjacency_matrix = AdjacencyMatrix()
proximal_cues = ProximalCues()
tfidf = TfIdf()

In [7]:
generic = lambda x: literal_eval(x)

conv = {'url_id_path': generic,
        'seconds_spent_path': generic}
df = pd.read_csv('../data/clickdata/dataNoUnscrapedVisitsOrUnder20Sec.csv', converters=conv)

paths = df.url_id_path

I took the liberty to implement my own - perhaps but not necessarily slightly simplified - version of IUNIS.

Input is the ordered list of page id's that the user has visited.
For each page, take the TF-IDF values from that page, which is a row of keywords, with either 0 if the keyword is not present or 0<value<=1 if it is.

Then, for each next page, we add the TF-IDF values onto the existing weights.

Eventually we can sort the list and we will have the keywords with the highest weight on top.

In [ ]:
from pandas import DataFrame
def find_keyword_weights(path):
    weights = np.zeros(tfidf.get_number_of_keywords())
    pd.options.display.max_rows = 0
    for id in path:
        weights += tfidf.get_all_keywords_by_id_normalized(id)

    df = DataFrame(weights, columns=['weights'])
    return df.sort_values(by=['weights'], ascending=False)

In this model, I introduced a factor of decay. Essentially, every iteration, all values are divided by 1.25, meaning that recent keywords are more biased.
This is a very simple implementation, we can experiment with changing this 1.25, and also look at different implementations.

In [ ]:
def find_keyword_weights_more_weight_on_recent_pages(path):
    weights = np.zeros(tfidf.get_number_of_keywords())
    pd.options.display.max_rows = 0
    for id in path:
        weights = weights/1.25
        weights += tfidf.get_all_keywords_by_id_normalized(id)

    df = DataFrame(weights, columns=['weights'])
    return df.sort_values(by=['weights'], ascending=False)

This is a method to print out the top 'num_of_words' weighted keywords.

In [8]:
def print_top_weights_as_words(num_of_words, weights):
    top_weights = weights.head(num_of_words)
    i=0
    for index in top_weights.index:
        print(str(tfidf.get_keyword_by_id(index)) + " : " + str(top_weights.weights.iloc[i]))
        i+=1
    print("\n")

Line 2 in this section gets rid of all useless pages (login, logout, search, and error)
Then, if the user has visited more than 5 pages, it will compute the keywords up until page 5.

In [ ]:
for path in paths:
    path[:] = [x for x in path if x < 1556]
    if len(path)>5:
        path[:] = path[:5]
        print_top_weights_as_words(10, find_keyword_weights(path))

These sections computes the weights after each different page to see how it changes throughout the visit
We can do interesting experiments to see how information need changes throughout the visit, and to find out at which page index we can best make our prediction final

In [33]:
path = paths[12]
path[:] = [x for x in path if x < 1556]
print(path)

[188, 228, 3, 1, 12, 7, 20, 1, 7, 1, 12, 7, 13, 188, 1]


In [34]:
for i in range(len(path)):
    subpath = path[:i+1]
    weights = find_keyword_weights_more_weight_on_recent_pages(subpath)
    print_top_weights_as_words(10, weights)

bijverdienen : 0.0838899052653509
gekregen : 0.07342690609120023
jaaropgave : 0.0723186300120255
betaalspecificatie : 0.06883927120918737
nabetaling : 0.06679300958519852
buitenland : 0.045383799679589665
dekkingsgraad : 0.04179540235127088
speciaal : 0.03628136290531201
uitbetaling : 0.03617637516726869
betaaldatums : 0.030811020135422404


nabestaande : 0.1296809887240472
nabestaandenpensioen : 0.10169550611141687
verander : 0.0736911189788815
bijverdienen : 0.06711192421228071
gekregen : 0.05874152487296018
jaaropgave : 0.0578549040096204
betaalspecificatie : 0.05507141696734989
nabetaling : 0.05343440766815881
verhoog : 0.052034322325171
regel : 0.04864181290583492


nabestaandenpensioen : 0.13035720664520836
samenvatting : 0.10899552751594592
kiezen : 0.10773159420182397
nabestaande : 0.10374479097923776
samenstellen : 0.10293099656999839
ineen : 0.084933645432089
deeltijdpensioen : 0.08270564389222383
loonheffingskorting : 0.07826937410655303
aanpassen : 0.07525841345119526
pensi

Idea for evaluation:

Look at the keywords at a specific time (we must make sure the weights are normalised, they are not in the implementation above), and compare it with a certain page near the end of the user visit to see if the user found their need.
We can do either the last page, or maybe the page that the user has spent the most time on.
The problem becomes that we want to evaluate the prediction of the information need, and there will be inaccuracies as the user data is not necessarily correct.
But if we can create a metric and we can compare different implementations with the same metric we still have something nice.


[1, 2]
